## 네이버 금융 데이터 크롤링 코드

In [1]:
import requests
from bs4 import BeautifulSoup
import traceback
import pandas as pd

In [2]:
# 원하는 기업의 코드를 알면 쉽게 얻을 수 있다.
code_num = ['005690','091090','006620','067630','078160','031390','003000','065660','144510','038290','217730','007390','118000','032980','203690','166480','271980','085660','191420','034940']

In [3]:
code_name = ['파미셀','세원셀론텍','동구바이오제약','에이치엘비생명과학','메디포스트','녹십자셀','부광약품','안트로젠','녹십자랩셀','마크로젠','강스템바이오텍','네이처셀','우리들휴브레인','바이온','프로스테믹스','코아스템','제일약품','차바이오텍','테코사이언스','조아제약']

In [4]:
def parse_page(code, page):
    try:
        url = 'http://finance.naver.com/item/sise_day.nhn?code={code}&page={page}'.format(code=code, page=page)
        res = requests.get(url)
        _soap = BeautifulSoup(res.text, 'lxml')
        _df = pd.read_html(str(_soap.find("table")), header=0)[0]
        _df = _df.dropna()
        return _df
    except Exception as e:
        traceback.print_exc()
    return None

In [5]:
#i는 위치, v는 code num
for i, v in enumerate(code_num):
    code = v
    url = 'http://finance.naver.com/item/sise_day.nhn?code={code}'.format(code=code)
    res = requests.get(url)
    res.encoding = 'utf-8'
    res.status_code
    
    soap = BeautifulSoup(res.text, 'lxml')
    
    
    el_table_navi = soap.find("table", class_="Nnavi")
    el_td_last = el_table_navi.find("td", class_="pgRR")
    pg_last = el_td_last.a.get('href').rsplit('&')[1]
    pg_last = pg_last.split('=')[1]
    pg_last = int(pg_last)
    
    data = parse_page(code, 1)
    
    for j in range(2,62):
        data = pd.concat([data,pd.DataFrame(parse_page(code, j))],axis = 0)
        
    #data.to_csv('%s.csv'%code_name[i])
    

In [6]:
data

,날짜,종가,전일비,시가,고가,저가,거래량
1,2020.12.22,5550.0,260.0,5330.0,5770.0,5200.0,13106468.0
2,2020.12.21,5290.0,210.0,5130.0,5450.0,5030.0,2794181.0
3,2020.12.18,5080.0,20.0,5100.0,5150.0,5020.0,400998.0
4,2020.12.17,5060.0,140.0,5240.0,5240.0,5060.0,777372.0
5,2020.12.16,5200.0,20.0,5240.0,5270.0,5120.0,797081.0
...,...,...,...,...,...,...,...
9,2018.07.10,4950.0,110.0,4840.0,4985.0,4830.0,168646.0
10,2018.07.09,4840.0,115.0,4710.0,4895.0,4650.0,295918.0
11,2018.07.06,4725.0,0.0,4725.0,4770.0,4555.0,216009.0
12,2018.07.05,4725.0,100.0,4830.0,4935.0,4660.0,283983.0
